## Alliance Against 3D Printed Guns

We are an alliance of global developers, hackers, data scientists, professors and general modern technology experts. We believe that legal 3D printed guns pose a threat to the people around the world and are working on open source Artificial Intelligence and 3D Printing technologies to help parents protect their children whilst they use the internet and assist website owners classify & remove unwanted gun designs from their websites.

Through the use of Artificial Intelligence we will provide free tools, services and source codes to those who want to protect themselves and the people around them from the dangers of open source 3D weapons.

# Create trainer job

The first step is to create a script that can be used to create a job on the A.I. DevCloud for training a Covolutional Neural Network created with Caffe. 

Before you run the following block make sure you have followed all of the steps at the beginning of the README file in the home directory of this project.

In [1]:
%%writefile AA3PG-Trainer
cd $PBS_O_WORKDIR
echo "* Hello world from compute server `hostname` on the A.I. DevCloud!"
echo "* The current directory is ${PWD}."
echo "* Compute server's CPU model and number of logical CPUs:"
lscpu | grep 'Model name\\|^CPU(s)'
echo "* Python available to us:"
export PATH=/glob/intel-python/python3/bin:$PATH;
which python
python --version 
echo "* Starting AA3PG-Trainer job"
echo "* This job trains the AA3PG 2D Image Classifier on the Colfax Cluster"
python Train.py
sleep 10
echo "*Adios"
# Remember to have an empty line at the end of the file; otherwise the last command will not run


Writing AA3PG-Trainer


# Check the job script was created

Now check that the job script was created successfully by executing the following block which will print out the files located in the current directory. If all was successful, you should see the file "AA3PG-Trainer". You can also open this file to confirm that the contents are correct.

In [2]:
%ls

AA3PG-Trainer  data/  Trainer.ipynb  Train.py


# Submit the job script

Now it is time to submit your training job script, this will queue the training script ready for execution and return your job ID. In this command we set the walltime to 24 hours, which should give our script enough time to fully complete without getting killed. 

In [3]:
!qsub -l walltime=24:00:00 AA3PG-Trainer

137316.c009


# Check the status of the job

Now you can monitor the status of the job by executing the following block. You may need to do this a number of times before the job completes. 

JOB STATUSES

R: Running  
Q: Waiting in queue

In [4]:
!qstat

Job ID                    Name             User            Time Use S Queue
------------------------- ---------------- --------------- -------- - -----
137314.c009                ...ub-singleuser u13339          00:00:01 R jupyterhub     
137316.c009                AA3PG-Trainer    u13339                 0 R batch          


You can also get a full list of stats for the job by executing the following block, replacing the ID with your job ID:

In [5]:
!qstat -f 137316

Job Id: 137316.c009
    Job_Name = AA3PG-Trainer
    Job_Owner = u13339@c009-n009
    job_state = R
    queue = batch
    server = c009
    Checkpoint = u
    ctime = Sat Aug  4 04:54:10 2018
    Error_Path = c009-n009:/home/u13339/AA3PG/AA3PG-Trainer.e137316
    exec_host = c009-n015/0-1
    Hold_Types = n
    Join_Path = n
    Keep_Files = n
    Mail_Points = n
    mtime = Sat Aug  4 04:54:11 2018
    Output_Path = c009-n009:/home/u13339/AA3PG/AA3PG-Trainer.o137316
    Priority = 0
    qtime = Sat Aug  4 04:54:10 2018
    Rerunable = True
    Resource_List.nodect = 1
    Resource_List.nodes = 1:ppn=2
    Resource_List.walltime = 24:00:00
    session_id = 159340
    Variable_List = PBS_O_QUEUE=batch,PBS_O_HOME=/home/u13339,
	PBS_O_LOGNAME=u13339,
	PBS_O_PATH=/glob/intel-python/python3/bin/:/glob/intel-python/python3
	/bin/:/glob/intel-python/python2/bin/:/glob/development-tools/versions
	/intel-parallel-studio-2018-update3/compilers_and_libraries_2018.3.222
	/linux/bin/intel64:/glob/d

# Check error and output files

After the above job finished you will see two files in your current directory, as the job ID in my case was 137316, my error file ends with e137316 and my output file ends with o137316. In this case the error file contained a FutureWarning. The output will show you the full output of your program.

In [10]:
%ls

AA3PG-Trainer          AA3PG-Trainer.o137316  Trainer.ipynb
AA3PG-Trainer.e137316  data/                  Train.py
